In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# load super bowl data
super_bowl_file = "Resources/Super_Bowl.csv"
super_bowl_df = pd.read_csv(super_bowl_file)
super_bowl_df.head()

,Date,SB,Attendance,QB Winner,Coach Winner,Winner,Winning Pts,QB Loser,Coach Loser,Loser,...,City,State,Point Difference,Referee,Umpire,Head Linesman,Line Judge,Field Judge,Back Judge,Side Judge
0,15-Jan-67,I,61946,Bart Starr,Vince Lombardi,Green Bay Packers,35,Len Dawson,Hank Stram,Kansas City Chiefs,...,Los Angeles,California,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14-Jan-68,II,75546,Bart Starr,Vince Lombardi,Green Bay Packers,33,Daryle Lamonica,John Rauch,Oakland Raiders,...,Miami,Florida,19,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12-Jan-69,III,75389,Joe Namath,Weeb Ewbank,New York Jets,16,"Earl Morrall, Johnny Unitas",Don Shula,Baltimore Colts,...,Miami,Florida,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11-Jan-70,IV,80562,"Len Dawson, Mike Livingston",Hank Stram,Kansas City Chiefs,23,Joe Kapp,Bud Grant,Minnesota Vikings,...,New Orleans,Louisiana,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,17-Jan-71,V,79204,"Earl Morrall , Johnny Unitas",Don McCafferty,Baltimore Colts,16,Craig Morton,Tom Landry,Dallas Cowboys,...,Miami,Florida,3,Norm Schachter,Paul Trepinski,Ed Marion,Jack Fette,Fritz Graf,Hugh Gamber,NaN


In [3]:
# clean sb df (make sb # index, drop unneccesary columns, concatenate city/state columns and rename as needed)
cleaned_sb_df = super_bowl_df.drop(["Date", "QB  Winner", "Coach Winner", "Coach Loser", "Referee", "Umpire", "Head Linesman", "Line Judge", "Field Judge", "Back Judge", "Side Judge", "Point Difference", "QB Loser"], axis =1)
cleaned_sb_df = cleaned_sb_df[["SB", "Attendance", "Winner", "Winning Pts", "Loser", "Losing Pts", "MVP", "Stadium", "City", "State"]]
cleaned_sb_df = cleaned_sb_df.rename(columns={"SB":"sb", "Attendance":"attendance", "Winner":"winner", "Winning Pts":"winning_pts", "Loser":"loser", "Losing Pts":"losing_pts", "MVP":"mvp", "Stadium":"stadium", "City":"city", "State":"state"})
cleaned_sb_df['city_state'] = cleaned_sb_df['city'] + [' '] + cleaned_sb_df['state']
cleaned_sb_df=cleaned_sb_df.drop(["city", "state"], axis=1)
cleaned_sb_df.reset_index(drop=True)
cleaned_sb_df.head()
cleaned_sb_df.head()

,sb,attendance,winner,winning_pts,loser,losing_pts,mvp,stadium,city_state
0,I,61946,Green Bay Packers,35,Kansas City Chiefs,10,Bart Starr,Memorial Coliseum,Los Angeles California
1,II,75546,Green Bay Packers,33,Oakland Raiders,14,Bart Starr,Orange Bowl,Miami Florida
2,III,75389,New York Jets,16,Baltimore Colts,7,Joe Namath,Orange Bowl,Miami Florida
3,IV,80562,Kansas City Chiefs,23,Minnesota Vikings,7,Len Dawson,Tulane Stadium,New Orleans Louisiana
4,V,79204,Baltimore Colts,16,Dallas Cowboys,13,Chuck Howley,Orange Bowl,Miami Florida


In [4]:
# load us cities needed for lat/long data 
us_cities_file = "Resources/uscities.csv"
us_cities_df = pd.read_csv(us_cities_file)
us_cities_df.head()

,city,city_ascii,state_id,state_name,county_fips,county_name,county_fips_all,county_name_all,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
0,South Creek,South Creek,WA,Washington,53053,Pierce,53053,Pierce,46.9994,-122.3921,2500,125.0,polygon,False,True,America/Los_Angeles,3,98580 98387 98338,1840042075
1,Roslyn,Roslyn,WA,Washington,53037,Kittitas,53037,Kittitas,47.2507,-121.0989,947,84.0,polygon,False,True,America/Los_Angeles,3,98941 98068 98925,1840019842
2,Sprague,Sprague,WA,Washington,53043,Lincoln,53043,Lincoln,47.3048,-117.9713,441,163.0,polygon,False,True,America/Los_Angeles,3,99032,1840021107
3,Gig Harbor,Gig Harbor,WA,Washington,53053,Pierce,53053,Pierce,47.3352,-122.5968,9507,622.0,polygon,False,True,America/Los_Angeles,3,98332 98335,1840019855
4,Lake Cassidy,Lake Cassidy,WA,Washington,53061,Snohomish,53061,Snohomish,48.0639,-122.0920,3591,131.0,polygon,False,True,America/Los_Angeles,3,98223 98258 98270,1840041959


In [5]:
# clean us cities df (drop unneccesary columns, concatenate city/state columns and rename as needed)
cleaned_us_cities_df = us_cities_df[["city", "state_name", "lat", "lng"]]
cleaned_us_cities_df = cleaned_us_cities_df.rename(columns={"state_name":"state"})
cleaned_us_cities_df['city_state'] = cleaned_us_cities_df['city'] + [' '] + cleaned_us_cities_df['state']
cleaned_us_cities_df=cleaned_us_cities_df.drop(["city", "state"], axis=1)
cleaned_us_cities_df = cleaned_us_cities_df[['city_state','lat','lng']]
cleaned_us_cities_df.head()
cleaned_us_cities_df.head()

,city_state,lat,lng
0,South Creek Washington,46.9994,-122.3921
1,Roslyn Washington,47.2507,-121.0989
2,Sprague Washington,47.3048,-117.9713
3,Gig Harbor Washington,47.3352,-122.5968
4,Lake Cassidy Washington,48.0639,-122.0920


In [6]:
merged_superbowl_df = pd.merge(cleaned_sb_df, cleaned_us_cities_df, on='city_state', how='left')
superbowl = merged_superbowl_df.dropna()
superbowl

,sb,attendance,winner,winning_pts,loser,losing_pts,mvp,stadium,city_state,lat,lng
0,I,61946,Green Bay Packers,35,Kansas City Chiefs,10,Bart Starr,Memorial Coliseum,Los Angeles California,34.1139,-118.4068
1,II,75546,Green Bay Packers,33,Oakland Raiders,14,Bart Starr,Orange Bowl,Miami Florida,25.7839,-80.2102
2,III,75389,New York Jets,16,Baltimore Colts,7,Joe Namath,Orange Bowl,Miami Florida,25.7839,-80.2102
3,IV,80562,Kansas City Chiefs,23,Minnesota Vikings,7,Len Dawson,Tulane Stadium,New Orleans Louisiana,30.0687,-89.9288
4,V,79204,Baltimore Colts,16,Dallas Cowboys,13,Chuck Howley,Orange Bowl,Miami Florida,25.7839,-80.2102
5,VI,81023,Dallas Cowboys,24,Miami Dolphins,3,Roger Staubach,Tulane Stadium,New Orleans Louisiana,30.0687,-89.9288
6,VII,90182,Miami Dolphins,14,Washington Redskins,7,Jake Scott,Memorial Coliseum,Los Angeles California,34.1139,-118.4068
7,VIII,71882,Miami Dolphins,24,Minnesota Vikings,7,Larry Csonka,Rice Stadium,Houston Texas,29.7869,-95.3905
8,IX,80997,Pittsburgh Steelers,16,Minnesota Vikings,6,Franco Harris,Tulane Stadium,New Orleans Louisiana,30.0687,-89.9288
9,X,80187,Pittsburgh Steelers,21,Dallas Cowboys,17,Lynn Swann,Orange Bowl,Miami Florida,25.7839,-80.2102


In [7]:
connection_string = "postgres:chicago1023@localhost:5432/Project_2"
engine = create_engine(f'postgresql://{connection_string}')

In [8]:
# Confirm tables
engine.table_names()

['superbowl']

In [9]:
superbowl.to_sql(name='superbowl', con=engine, if_exists='append', index=False)

IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "superbowl_pkey"
DETAIL:  Key (sb)=(I) already exists.

[SQL: INSERT INTO superbowl (sb, attendance, winner, winning_pts, loser, losing_pts, mvp, stadium, city_state, lat, lng) VALUES (%(sb)s, %(attendance)s, %(winner)s, %(winning_pts)s, %(loser)s, %(losing_pts)s, %(mvp)s, %(stadium)s, %(city_state)s, %(lat)s, %(lng)s)]
[parameters: ({'sb': 'I', 'attendance': 61946, 'winner': 'Green Bay Packers', 'winning_pts': 35, 'loser': 'Kansas City Chiefs', 'losing_pts': 10, 'mvp': 'Bart Starr', 'stadium': 'Memorial Coliseum', 'city_state': 'Los Angeles California', 'lat': 34.1139, 'lng': -118.4068}, {'sb': 'II', 'attendance': 75546, 'winner': 'Green Bay Packers', 'winning_pts': 33, 'loser': 'Oakland Raiders', 'losing_pts': 14, 'mvp': 'Bart Starr', 'stadium': 'Orange Bowl', 'city_state': 'Miami Florida', 'lat': 25.7839, 'lng': -80.2102}, {'sb': 'III', 'attendance': 75389, 'winner': 'New York Jets', 'wi